# 🎓 Travaux Pratiques : StyleGAN (Génération Contrôlée Simplifiée)

**Auteur :**Benlahmar Habib


## Objectifs Pédagogiques

1.  **Mapping Network :** Comprendre l'avantage de séparer l'espace latent initial ($z$) de l'espace de style intermédiaire ($w$).
2.  **Adaptive Instance Normalization (AdaIN) :** Implémenter et comprendre comment cette couche module le style et la texture.
3.  **Contrôlabilité :** Observer comment le style est appliqué localement à chaque niveau de résolution du Générateur.
4.  **Base Stabilité :** Utiliser les principes du WGAN-GP pour garantir la stabilité de ce nouveau Générateur.

--- 

## I. Fondements Théoriques : Style et Séparation

Les GANs précédents souffraient de l'*enchevêtrement latent* : une dimension du bruit $z$ pouvait affecter simultanément la forme, la couleur et la texture. Le StyleGAN résout ce problème en introduisant un **Mapping Network** pour créer un espace de style ($w$) *dé-enchevêtré* et en injectant ce style via **AdaIN** à chaque couche.

### 1.1. Adaptive Instance Normalization (AdaIN)

AdaIN remplace la Batch Normalization dans le Générateur. Pour une activation $\mathbf{x}$ et un vecteur de style $w$ (qui fournit les paramètres de mise à l'échelle $\mathbf{y}_s$ et de décalage $\mathbf{y}_b$), l'opération est :

$$\text{AdaIN}(\mathbf{x}, \mathbf{y}) = \mathbf{y}_s \frac{\mathbf{x} - \mu(\mathbf{x})}{\sigma(\mathbf{x})} + \mathbf{y}_b$$

### Question d'Accompagnement (Q1.1)

En quoi l'utilisation de l'**Instance Normalization** (qui normalise chaque échantillon indépendamment du lot, $\mu(\mathbf{x}), \sigma(\mathbf{x})$) au lieu de la **Batch Normalization** (qui utilise $\mu(\text{batch}), \sigma(\text{batch})$) est-elle essentielle pour que la couche AdaIN puisse injecter le style de manière localisée et efficace ?

--- 

## II. Configuration et Architecture StyleGAN Simplifiée

Nous utilisons le WGAN-GP pour la perte du Critique et définissons les nouveaux modules du Générateur : AdaIN, Mapping Network et Synthesis Network.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from matplotlib import pyplot as plt
from torchvision.utils import make_grid
from tqdm.notebook import trange, tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Exécution sur {device}")

# Hyperparamètres WGAN-GP (Réutilisés)
latent_dim = 100 # Dimension de z (bruit initial)
style_dim = 100  # Dimension de w (style)
batch_size = 128
epochs = 100
lr = 0.0001
beta1 = 0.5
lambda_gp = 10 
n_critic = 5 
ngf = 64 # Nombre de feature maps du générateur
ndf = 64 # Nombre de feature maps du discriminateur

# NORMALISATION : [0, 1] -> [-1, 1]
transform = transforms.Compose([
    transforms.ToTensor(), 
    transforms.Normalize((0.5,), (0.5,)) 
])

train_dataset = datasets.FashionMNIST(root='./data/FashionMNIST', train=True, download=True, transform=transform)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# --- 2.1. Module AdaIN ---
class AdaptiveInstanceNorm(nn.Module):
    def __init__(self, num_features, style_dim):
        super().__init__()
        # Projette le style w en paramètres y_s et y_b
        self.style_scale = nn.Linear(style_dim, num_features)
        self.style_bias = nn.Linear(style_dim, num_features)

        self.instance_norm = nn.InstanceNorm2d(num_features, affine=False) 

    def forward(self, x, w):
        # Calcul des paramètres de style, formatés pour l'opération (1, C, 1, 1)
        y_s = self.style_scale(w).unsqueeze(-1).unsqueeze(-1)
        y_b = self.style_bias(w).unsqueeze(-1).unsqueeze(-1)
        
        # Applique l'Instance Normalization et le Style (y_s * x + y_b)
        x = self.instance_norm(x)
        return y_s * x + y_b


Exécution sur cpu


### 2.2. Mapping Network ($M$) et Synthesis Network ($G_{\text{synth}}$)

Le **Mapping Network** crée le style $w$. Le **Synthesis Network** utilise ce style pour construire l'image.

In [ ]:
class MappingNetwork(nn.Module):
    def __init__(self, latent_dim, style_dim, num_layers=4):
        super().__init__()
        layers = []
        in_dim = latent_dim
        for i in range(num_layers):
            layers.append(nn.Linear(in_dim, style_dim))
            layers.append(nn.LeakyReLU(0.2))
            in_dim = style_dim
        self.model = nn.Sequential(*layers)
        
    def forward(self, z):
        return self.model(z)

class SynthesisBlock(nn.Module):
    def __init__(self, in_channels, out_channels, style_dim, upsample=True):
        super().__init__()
        # Simplification : utilise nn.Upsample + Conv au lieu de ConvTranspose2d
        self.upsample = nn.Upsample(scale_factor=2, mode='nearest') if upsample else None
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1)
        self.adain = AdaptiveInstanceNorm(out_channels, style_dim)
        self.lrelu = nn.LeakyReLU(0.2)

    def forward(self, x, w):
        if self.upsample: x = self.upsample(x)
        x = self.conv(x)
        x = self.adain(x, w)
        x = self.lrelu(x)
        return x

class Generator(nn.Module):
    def __init__(self, latent_dim, style_dim):
        super().__init__()
        self.mapping = MappingNetwork(latent_dim, style_dim)
        
        # Input constant (remplace l'entrée spatiale du bruit)
        self.const_input = nn.Parameter(torch.randn(1, ngf * 4, 4, 4))
        
        self.synth = nn.Sequential(
            SynthesisBlock(ngf * 4, ngf * 4, style_dim, upsample=False), # 4x4
            SynthesisBlock(ngf * 4, ngf * 2, style_dim, upsample=True), # 8x8 
            SynthesisBlock(ngf * 2, ngf, style_dim, upsample=True), # 16x16
            SynthesisBlock(ngf, ngf, style_dim, upsample=True), # 32x32
        )
        
        self.to_rgb = nn.Conv2d(ngf, 1, kernel_size=1) 
        self.tanh = nn.Tanh()

    def forward(self, z):
        w = self.mapping(z) # Style vector
        
        # Répéter la constante pour le batch
        x = self.const_input.repeat(z.size(0), 1, 1, 1) 
        
        # Passage dans le Synthesis Network
        for module in self.synth:
            if isinstance(module, SynthesisBlock):
                x = module(x, w) 
            else:
                x = module(x)
        
        x = self.to_rgb(x)
        # Cropping 32x32 -> 28x28 pour Fashion-MNIST (pour simplifier le réseau)
        x = x[:, :, 2:30, 2:30] 
        return self.tanh(x)


### Question d'Accompagnement (Q2.1)

Dans l'architecture StyleGAN, l'entrée bruit $z$ n'est utilisée que dans le **Mapping Network** pour créer le style $w$. Quelle est l'autre source de bruit introduite directement dans le **Synthesis Network** (dans le StyleGAN complet) ? Quel est l'objectif de ce second bruit ?

--- 

## III. Boucle d'Entraînement (WGAN-GP pour la Stabilité)

Nous utilisons le Critique DCGAN et la perte WGAN-GP pour la stabilité.

In [ ]:
class Critic(nn.Module):
    def __init__(self):
        super().__init__()
        self.main = nn.Sequential(
            nn.Conv2d(1, ndf, 4, 2, 1, bias=True),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=True), 
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(ndf * 2, ndf * 4, 3, 2, 1, bias=True), 
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(ndf * 4, 1, 4, 1, 0, bias=True), 
        )

    def forward(self, input):
        return self.main(input).view(-1, 1)

# Fonction de Pénalité de Gradient (réutilisée)
def calculate_gradient_penalty(C, real_images, fake_images, lambda_gp, device):
    b_size = real_images.size(0)
    alpha = torch.rand(b_size, 1, 1, 1, device=device)
    x_hat = alpha * real_images + (1 - alpha) * fake_images
    x_hat.requires_grad_(True)
    C_x_hat = C(x_hat)

    gradients = torch.autograd.grad(outputs=C_x_hat,
                                    inputs=x_hat,
                                    grad_outputs=torch.ones_like(C_x_hat),
                                    create_graph=True,
                                    retain_graph=True)[0]

    gradients = gradients.view(b_size, -1)
    gradient_norm = gradients.norm(2, dim=1)
    gp = lambda_gp * ((gradient_norm - 1)**2).mean()
    return gp

# Initialisation
G = Generator(latent_dim, style_dim).to(device)
C = Critic().to(device)

G_optimizer = optim.Adam(G.parameters(), lr=lr, betas=(beta1, 0.999))
C_optimizer = optim.Adam(C.parameters(), lr=lr, betas=(beta1, 0.999))

fixed_noise = torch.randn(64, latent_dim, device=device)

# Fonction de visualisation
def show_grid(grid, title="", figsize=(10, 10)):
    plt.figure(figsize=figsize)
    plt.title(title)
    grid = (grid + 1) / 2 
    plt.imshow(np.transpose(grid.numpy(), (1, 2, 0)), cmap="gray")
    plt.axis("off")
    plt.show()

def train_stylegan_wgangp(G, C, G_optimizer, C_optimizer, dataloader, epochs, latent_dim, device, lambda_gp, n_critic):
    
    for epoch in trange(epochs, desc="Entraînement StyleGAN/WGAN-GP"):
        for i, (real_images, _) in enumerate(dataloader):
            
            real_images = real_images.to(device)
            b_size = real_images.size(0)
            
            # (1) Mise à jour C (Critique) : n_critic fois
            for _ in range(n_critic):
                C_optimizer.zero_grad()
                
                noise = torch.randn(b_size, latent_dim, device=device)
                fake_images = G(noise).detach() 
                
                # Loss WGAN
                C_real = C(real_images)
                C_fake = C(fake_images)
                W_distance = C_real.mean() - C_fake.mean()
                C_W_loss = -W_distance 
                
                # Pénalité de Gradient
                gp = calculate_gradient_penalty(C, real_images, fake_images, lambda_gp, device)
                C_loss = C_W_loss + gp
                
                C_loss.backward()
                C_optimizer.step()
                
            # (2) Mise à jour G (Générateur) : 1 fois
            G_optimizer.zero_grad()
            
            noise = torch.randn(b_size, latent_dim, device=device)
            fake_images = G(noise)
            C_fake = C(fake_images)
            
            G_loss = -C_fake.mean()
            
            G_loss.backward()
            G_optimizer.step()
            
        tqdm.write(f"Epoch {epoch+1:2d} | C Loss: {C_loss.item():.4f} | W Dist: {W_distance.item():.4f} | G Loss: {G_loss.item():.4f}")
        
        # 3. Visualisation
        if (epoch + 1) % 5 == 0:
            G.eval()
            with torch.no_grad():
                generated_images = G(fixed_noise).cpu()
                show_grid(make_grid(generated_images, 8), title=f"StyleGAN Génération Époque {epoch+1}")
            G.train()

# train_stylegan_wgangp(G, C, G_optimizer, C_optimizer, train_dataloader, epochs, latent_dim, device, lambda_gp, n_critic) # <-- DÉCOMMENTER POUR LANCER L'ENTRAINEMENT

--- 

## V. Synthèse et Ouverture (Post-Entraînement)

### Questions Finales

1.  **AdaIN vs. BN :** Expliquez la principale différence entre la Batch Normalization (BN) utilisée dans le DCGAN et l'Adaptive Instance Normalization (AdaIN) utilisée ici. Quel type d'information chaque méthode normalise-t-elle, et pourquoi AdaIN permet-il d'injecter des *styles* ?
2.  **Encêtrement Latent :** Quel problème théorique majeur le **Mapping Network** ($z \to w$) tente-t-il de résoudre par rapport à l'utilisation directe de $z$ comme entrée ? Quel est le lien avec la facilité de la manipulation sémantique des images ?
3.  **Hétérogénéité des Styles :** Le StyleGAN complet injecte différents vecteurs de style ($w_1, w_2, ...$) à différentes résolutions du réseau. Expliquez comment l'injection de $w_{\text{basse résolution}}$ peut influencer la *forme* générale de l'objet, tandis que $w_{\text{haute résolution}}$ influence la *texture* et les *détails*.
4.  **Au-delà de StyleGAN :** Le StyleGAN a été suivi par StyleGAN2 et StyleGAN3. Citez une des principales améliorations du StyleGAN2 (Indice : Normalisation et artefacts de gouttelette).